In [1]:
import joblib
import numpy as np
import pandas as pd

In [6]:
model    = joblib.load('stroke_model.pkl')
scaler   = joblib.load('stroke_scaler.pkl')
encoders = joblib.load('stroke_encoders.pkl')





In [7]:
def predict_stroke(gender, age, hypertension, heart_disease, ever_married,
                   work_type, residence_type, avg_glucose_level, bmi, smoking_status,
                   pretty=False):

    # Encode categorical variables
    try:
        gender_enc = encoders['gender'].transform([gender])[0]
        ever_married_enc = encoders['ever_married'].transform([ever_married])[0]
        work_type_enc = encoders['work_type'].transform([work_type])[0]
        residence_enc = encoders['Residence_type'].transform([residence_type])[0]
        smoking_enc = encoders['smoking_status'].transform([smoking_status])[0]
    except ValueError as e:
        return {"error": f"Invalid category value: {e}"}

    # Create DataFrame with proper column names and order
    input_data = pd.DataFrame(
        [[gender_enc, age, hypertension, heart_disease, ever_married_enc,
          work_type_enc, residence_enc, avg_glucose_level, bmi, smoking_enc]],
        columns=['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
                 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']
    )

    # Scale the input
    input_scaled = scaler.transform(input_data)

    # Make prediction
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0]

    result = {
        'prediction': int(prediction),
        'stroke_probability': float(round(probability[1] * 100, 2)),
        'no_stroke_probability': float(round(probability[0] * 100, 2)),
        'result': 'Stroke Risk' if prediction == 1 else 'No Stroke Risk'
    }

    if pretty:
        print("=" * 60)
        print("        STROKE PREDICTION RESULT")
        print("=" * 60)
        print(f"\n  Patient Data:")
        print(f"    Gender              : {gender}")
        print(f"    Age                 : {age}")
        print(f"    Hypertension        : {'Yes' if hypertension else 'No'}")
        print(f"    Heart Disease       : {'Yes' if heart_disease else 'No'}")
        print(f"    Ever Married        : {ever_married}")
        print(f"    Work Type           : {work_type}")
        print(f"    Residence           : {residence_type}")
        print(f"    Avg Glucose Level   : {avg_glucose_level}")
        print(f"    BMI                 : {bmi}")
        print(f"    Smoking Status      : {smoking_status}")
        print(f"\n  Prediction:")
        if result['prediction'] == 1:
            print(f"    🔴 {result['result']}")
        else:
            print(f"    🟢 {result['result']}")
        print(f"    Stroke Risk         : {result['stroke_probability']}%")
        print(f"    No Stroke Risk      : {result['no_stroke_probability']}%")
        print("=" * 60)
        return None

    return result

In [10]:
predict_stroke(
    gender='Female',
    age=80,
    hypertension=1,
    heart_disease=1,
    ever_married='Yes',
    work_type='Private',
    residence_type='Urban',
    avg_glucose_level=90.0,
    bmi=33,
    smoking_status='smokes',
    pretty=True
)

        STROKE PREDICTION RESULT

  Patient Data:
    Gender              : Female
    Age                 : 80
    Hypertension        : Yes
    Heart Disease       : Yes
    Ever Married        : Yes
    Work Type           : Private
    Residence           : Urban
    Avg Glucose Level   : 90.0
    BMI                 : 33
    Smoking Status      : smokes

  Prediction:
    🔴 Stroke Risk
    Stroke Risk         : 84.25%
    No Stroke Risk      : 15.75%
